# LDstore

Check tutorial [here](http://www.christianbenner.com/#)


# Installation

```
pip3 install https://files.pythonhosted.org/packages/a8/fd/f98ab7dea176f42cb61b80450b795ef19b329e8eb715b87b0d13c2a0854d/ldstore-0.1.9.tar.gz 
    ```

# Create master file

The master file is a semicolon-separated text file and contains no space. It contains the following mandatory column names and one dataset per line.

For the Z file modify this file to be rsid:chrom:pos:a1:a2. Formatting for chromosome should be 01,02,03...etc

`~/UKBiobank_Yale_transfer/ukb39554_imputeddataset/ukb_mfi_chr9_v3.txt`

For the sample files remember to use only unrelated individuals from the 500K genotyped participants

## Create the Z file

This file is created from the mfi files. To check the column description please look [here](https://biobank.ndph.ox.ac.uk/ukb/refer.cgi?id=531)

The z-file for the 1725 blocks needs to be created from the variant_list file stored here `/mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/113022_regions_bgen`

In [65]:
import pandas as pd
file = "/mnt/vast/hpc/csg/UKBiobank_Yale_transfer/ukb39554_imputeddataset/ukb_mfi_chr1_v3.txt"
z_file = pd.read_csv(file,header=None,sep="\t", names=["alternate_id", "rsid", "position", "allele1", "allele2", "maf", "minor_allele", "info_score"])

In [66]:
z_file.head()

,alternate_id,rsid,position,allele1,allele2,maf,minor_allele,info_score
0,1:10177_A_AC,rs367896724,10177,A,AC,0.400790,AC,0.467935
1,1:10235_T_TA,rs540431307,10235,T,TA,0.000367,TA,0.214688
2,1:10352_T_TA,rs201106462,10352,T,TA,0.394625,TA,0.447895
3,1:10505_A_T,rs548419688,10505,A,T,0.000018,T,0.230206
4,1:10506_C_G,rs568405545,10506,C,G,0.000018,G,0.230206


In [67]:
z_copy = z_file.copy()
z_copy=z_copy.join(z_copy['alternate_id'].str.split(':', 1, expand=True).rename(columns={0:'chromosome', 1:'other'}))

In [68]:
z_copy.head()

,alternate_id,rsid,position,allele1,allele2,maf,minor_allele,info_score,chromosome,other
0,1:10177_A_AC,rs367896724,10177,A,AC,0.400790,AC,0.467935,1,10177_A_AC
1,1:10235_T_TA,rs540431307,10235,T,TA,0.000367,TA,0.214688,1,10235_T_TA
2,1:10352_T_TA,rs201106462,10352,T,TA,0.394625,TA,0.447895,1,10352_T_TA
3,1:10505_A_T,rs548419688,10505,A,T,0.000018,T,0.230206,1,10505_A_T
4,1:10506_C_G,rs568405545,10506,C,G,0.000018,G,0.230206,1,10506_C_G


In [69]:
z_copy["chromosome"]=z_copy['chromosome'].str.zfill(2)

In [70]:
z_copy=z_copy.drop(columns=['other'])

In [74]:
z_copy.head()

,alternate_id,rsid,position,allele1,allele2,maf,minor_allele,info_score,chromosome
0,1:10177_A_AC,rs367896724,10177,A,AC,0.400790,AC,0.467935,01
1,1:10235_T_TA,rs540431307,10235,T,TA,0.000367,TA,0.214688,01
2,1:10352_T_TA,rs201106462,10352,T,TA,0.394625,TA,0.447895,01
3,1:10505_A_T,rs548419688,10505,A,T,0.000018,T,0.230206,01
4,1:10506_C_G,rs568405545,10506,C,G,0.000018,G,0.230206,01


In [72]:
basename = os.path.basename(file)

In [75]:
basename + '.z'

'ukb_mfi_chr1_v3.txt.z'

In [76]:
z_copy[["rsid","chromosome", "position", "allele1", "allele2", "alternate_id", "minor_allele", "maf", "info_score"]].to_csv('/mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/111822_LDstore_files/ukb39554_imputeddataset/' + basename + ".z" , sep='\t', index=False, header=True)

## 1725 Ld blocks

In [1]:
import pandas as pd
file = "/mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/113022_regions_bgen/ukb_imp_chr22_v3.22:16050408-17674295.variants"
z_file = pd.read_csv(file,header=0,sep="\t", skiprows=1, skipfooter=1, names=['alternate_ids', 'rsid', 'chromosome', 'position', 'number_of_alleles', 'allele1', 'allele2'])

/tmp/3191082.1.csg2.q/ipykernel_58042/2685489524.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  from collections import Sized, KeysView, Sequence
/home/dmc2245/miniconda3/lib/python3.8/site-packages/sos_notebook/kernel.py:1334: DeprecationWarning: Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  msg['msg_id'] = self._parent_header['header']['msg_id']


In [2]:
z_file

,alternate_ids,rsid,chromosome,position,number_of_alleles,allele1,allele2
0,22:16050435_T_C,22:16050435_T_C,22,16050435,2,T,C
1,22:16050527_C_A,rs587769434,22,16050527,2,C,A
2,22:16050568_C_A,rs587638893,22,16050568,2,C,A
3,22:16050607_G_A,rs587720402,22,16050607,2,G,A
4,22:16050627_G_T,rs587593704,22,16050627,2,G,T
...,...,...,...,...,...,...,...
38539,rs4819973,rs4819973,22,17674117,2,C,T
38540,22:17674167_C_A,rs182660568,22,17674167,2,C,A
38541,22:17674283_G_A,rs185848239,22,17674283,2,G,A
38542,22:17674284_C_T,rs558308473,22,17674284,2,C,T


In [3]:
chr22 = "/mnt/vast/hpc/csg/UKBiobank_Yale_transfer/ukb39554_imputeddataset/ukb_mfi_chr22_v3.txt"

In [4]:
chr22_file = pd.read_csv(chr22,header=None,sep="\t", names=["alternate_id", "rsid", "position", "allele1", "allele2", "maf", "minor_allele", "info_score"])

In [5]:
chr22_file

,alternate_id,rsid,position,allele1,allele2,maf,minor_allele,info_score
0,22:16050075_A_G,rs587697622,16050075,A,G,0.000019,G,0.081305
1,22:16050115_G_A,rs587755077,16050115,G,A,0.006524,A,0.192100
2,22:16050213_C_T,rs587654921,16050213,C,T,0.002442,T,0.188760
3,22:16050319_C_T,rs587712275,16050319,C,T,0.000573,T,0.229548
4,22:16050435_T_C,22:16050435_T_C,16050435,T,C,0.000586,C,0.292899
...,...,...,...,...,...,...,...,...
1255678,22:51241342_C_A,rs546768261,51241342,C,A,0.000012,A,0.229709
1255679,22:51241386_C_G,rs568168135,51241386,C,G,0.009836,G,0.394658
1255680,22:51244163_A_G,rs199560686,51244163,A,G,0.000143,G,0.458772
1255681,22:51244205_C_T,rs556942236,51244205,C,T,0.000004,T,0.005019


In [6]:
subset=chr22_file.join(z_file.set_index(['alternate_ids', 'rsid', 'position', 'allele1', 'allele2']), on=['alternate_id', 'rsid', 'position', 'allele1', 'allele2'], how='right')

In [7]:
subset

,alternate_id,rsid,position,allele1,allele2,maf,minor_allele,info_score,chromosome,number_of_alleles
4,22:16050435_T_C,22:16050435_T_C,16050435,T,C,0.000586,C,0.292899,22,2
5,22:16050527_C_A,rs587769434,16050527,C,A,0.001291,A,0.622228,22,2
6,22:16050568_C_A,rs587638893,16050568,C,A,0.000041,A,0.087629,22,2
7,22:16050607_G_A,rs587720402,16050607,G,A,0.000492,A,0.347865,22,2
8,22:16050627_G_T,rs587593704,16050627,G,T,0.000022,T,0.092050,22,2
...,...,...,...,...,...,...,...,...,...,...
38543,rs4819973,rs4819973,17674117,C,T,0.270722,T,1.000000,22,2
38544,22:17674167_C_A,rs182660568,17674167,C,A,0.000728,A,0.471369,22,2
38545,22:17674283_G_A,rs185848239,17674283,G,A,0.000314,A,0.779745,22,2
38546,22:17674284_C_T,rs558308473,17674284,C,T,0.000019,T,0.427082,22,2


In [9]:
rslt_df = subset[subset['maf'] > 0.1]

In [10]:
rslt_df

,alternate_id,rsid,position,allele1,allele2,maf,minor_allele,info_score,chromosome,number_of_alleles
18,22:16050822_G_A,rs12172168,16050822,G,A,0.149999,A,0.394574,22,2
35,22:16051249_T_C,rs62224609,16051249,T,C,0.100842,C,0.969322,22,2
57,22:16052080_G_A,rs4965031,16052080,G,A,0.126859,A,0.246037,22,2
65,22:16052167_A_AAAAC,rs375684679,16052167,A,AAAAC,0.438689,AAAAC,0.252585,22,2
118,22:16053659_A_C,rs915675,16053659,A,C,0.187545,A,0.275816,22,2
...,...,...,...,...,...,...,...,...,...,...
38392,22:17670256_C_T,rs1544504,17670256,C,T,0.473235,C,0.982634,22,2
38435,Affx-19036290,rs2286348,17671133,C,G,0.492970,C,1.000000,22,2
38523,22:17673719_GA_G,22:17673719_GA_G,17673719,GA,G,0.498035,G,0.956246,22,2
38543,rs4819973,rs4819973,17674117,C,T,0.270722,T,1.000000,22,2


In [11]:
rslt_df['alternate_id']

18           22:16050822_G_A
35           22:16051249_T_C
57           22:16052080_G_A
65       22:16052167_A_AAAAC
118          22:16053659_A_C
                ...         
38392        22:17670256_C_T
38435          Affx-19036290
38523       22:17673719_GA_G
38543              rs4819973
38547        22:17674295_G_A
Name: alternate_id, Length: 2709, dtype: object

In [15]:
rslt_df[["alternate_id"]].to_csv('/mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/111822_LDstore_files/ukb_imp_chr22_v3.22:16050408-17674295.variants.mafabove0.01' , sep=' ', index=False, header=False)

In [13]:
rslt_df[["rsid","chromosome", "position", "allele1", "allele2", "alternate_id"]].to_csv('/mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/111822_LDstore_files/ukb_imp_chr22_v3.22:16050408-17674295.variants.mafabove0.01.z' , sep=' ', index=False, header=True)

### Automating creation of the z files

In [ ]:
import glob, os
import pandas as pd
# assign directory
flist=glob.glob("/mnt/vast/hpc/csg/UKBiobank_Yale_transfer/ukb39554_imputeddataset/ukb_mfi_chr*_v3.txt")
for filename in flist:
    print(filename)
    basename=os.path.basename(filename)
    print(basename)
    file=pd.read_csv(filename,header=None,sep="\t", names=["alternate_id", "rsid", "position", "allele1", "allele2", "maf", "minor_allele", "info_score"])
    f_copy=file.copy()
    f_copy.head()
    f_copy=f_copy.join(f_copy['alternate_id'].str.split(':', 1, expand=True).rename(columns={0:'chromosome', 1:'other'}))
    f_copy=f_copy.drop(columns=['other'])
    f_copy["chromosome"]=f_copy['chromosome'].str.zfill(2)
    f_copy[["rsid","chromosome", "position", "allele1", "allele2", "alternate_id", "minor_allele", "maf", "info_score"]].to_csv("/mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/111822_LDstore_files/ukb39554_imputeddataset/" + basename + ".z", sep=" ", index=False, header=True)

### Create the z files with the subset of variants in a region

Additionally if you want to filter for MAF this needs to be done using the mfi file for the UKB case and providing the variants that you would like to analyze in the z file. 

In [ ]:
import glob, os
import pandas as pd
# assign directory
regions=glob.glob("/mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/113022_regions_bgen/ukb_imp_chr*.variants")
mfi_files=glob.glob("/mnt/vast/hpc/csg/UKBiobank_Yale_transfer/ukb39554_imputeddataset/ukb_mfi_chr*_v3.txt")
for filename in regions:
    print(filename)
    basename=os.path.basename(filename)
    sp = basename.split("_")
    chrom = [c for c in sp if "chr" in c][0]
    print(basename)
    fullregion = pd.read_csv(filename,header=0,sep="\t", skiprows=1, skipfooter=1, names=['alternate_ids', 'rsid', 'chromosome', 'position', 'number_of_alleles', 'allele1', 'allele2'])
    print('The number of variants in the region is:',fullregion.shape[0])
    mfi_file = [file for file in mfi_files if chrom in file][0]
    chrom_file = pd.read_csv(mfi_file,header=None,sep="\t", names=["alternate_id", "rsid", "position", "allele1", "allele2", "maf", "minor_allele", "info_score"])
    print('The number of variants in the mfi file is:',chorm_file.shape[0])
    subset_region=chrom_file.join(fullregion.set_index(['alternate_ids', 'rsid', 'position', 'allele1', 'allele2']), on=['alternate_id', 'rsid', 'position', 'allele1', 'allele2'], how='right')
    subset_maf=subset_region[subset_region['maf'] > 0.01]
    print('The number of variants in the filtered z_file is:',subset_maf.shape[0])
    subset_maf[["rsid","chromosome", "position", "allele1", "allele2", "alternate_ids"]].to_csv('/mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/111822_LDstore_files/121322_regions/' + basename + ".z" , sep='\t', index=False, header=True)

In [ ]:
### Creation of the masterfile
import pandas as pd
from datetime import datetime
date = datetime.now().strftime('%Y%m%d')
cwd='/mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/111822_LDstore_files/121322_regions/'
z_files = sorted(glob.glob({cwd}+'*.z'), key=len)
df = pd.DataFrame({'z':z_files})
bgen=sorted(glob.glob({cwd}+'*.bgen'), key=len)
df = pd.DataFrame({'bgen':bgen})
bgi=sorted(glob.glob({cwd}+'*.bgen.bgi'), key=len)
df = pd.DataFrame({'bgi':bgi})
bcor=[i.replace('bgen','bcor') for i in bcor]
df = pd.DataFrame({'bcor':bcor})
ld=[i.replace('bgen','ld') for i in ld]
df = pd.DataFrame({'ld':ld})
print(df)
#Add a constant number to every row in the sample column (the number of samples to analyze)
result['n_samples'] = df.apply(lambda x:487409, axis=1)
result.to_csv({cwd}+'masterfile_'+ {date}, sep=";", index=False, header=True)

# Test using the alternate_id instead of rsid

First of all use plink2 to read bgen file and output the snpids to a file to compare with mfi file. Please read this [file](https://biobank.ndph.ox.ac.uk/ukb/refer.cgi?id=531) to understand why ref-first

In [3]:
module load PLINK/2.0
plink2 --bgen /mnt/vast/hpc/csg/UKBiobank_Yale_transfer/ukb39554_imputeddataset/ukb_imp_chr22_v3.bgen 'ref-first' \
       --sample /mnt/vast/hpc/csg/UKBiobank_Yale_transfer/ukb39554_imputeddataset/ukb32285_imputedindiv.sample \
       --write-snplist \
       --out /mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/111822_LDstore_files/chr22_snplist

PLINK v2.00a2.3LM 64-bit Intel (24 Jan 2020)   www.cog-genomics.org/plink/2.0/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/111822_LDstore_files/chr22_snplist.log.
Options in effect:
  --bgen /mnt/vast/hpc/csg/UKBiobank_Yale_transfer/ukb39554_imputeddataset/ukb_imp_chr22_v3.bgen ref-first
  --out /mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/111822_LDstore_files/chr22_snplist
  --sample /mnt/vast/hpc/csg/UKBiobank_Yale_transfer/ukb39554_imputeddataset/ukb32285_imputedindiv.sample
  --write-snplist

Start time: Tue Nov 22 11:13:31 2022
257482 MiB RAM detected; reserving 128741 MiB for main workspace.
Allocated 72416 MiB successfully, after larger attempt(s) failed.
Using up to 64 threads (change this with --threads).
--bgen: 1255683 variants detected, format v1.2.
487409 samples imported from .sample file to
/mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/111822_LDstore_fi

In [17]:
import pandas as pd
file= '/mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/111822_LDstore_files/ukb39554_imputeddataset/ukb_mfi_chr22_v3.txt.z'
z = pd.read_csv(file,header=0,sep=" ")

In [18]:
z.head()

,rsid,chromosome,position,allele1,allele2,alternate_id,minor_allele,maf,info_score
0,rs587697622,22,16050075,A,G,22:16050075_A_G,G,0.000019,0.081305
1,rs587755077,22,16050115,G,A,22:16050115_G_A,A,0.006524,0.192100
2,rs587654921,22,16050213,C,T,22:16050213_C_T,T,0.002442,0.188760
3,rs587712275,22,16050319,C,T,22:16050319_C_T,T,0.000573,0.229548
4,22:16050435_T_C,22,16050435,T,C,22:16050435_T_C,C,0.000586,0.292899


In [10]:
file= '/mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/111822_LDstore_files/chr22_snplist.snplist'
snplist = pd.read_csv(file,header=None,sep=" ", names=["alternate_id"])

In [19]:
 z[["alternate_id","chromosome", "position", "allele1", "allele2", "rsid", "minor_allele", "maf", "info_score"]].to_csv("/mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/111822_LDstore_files/chr22_snplist_alternate_id.z", sep=" ", index=False, header=True)

In [19]:
snplist.head()

,alternate_id
0,rs587697622
1,rs587755077
2,rs587654921
3,rs587712275
4,22:16050435_T_C


In [25]:
z.shape

(1255683, 9)

In [26]:
snplist.shape

(1255683, 1)

In [27]:
z['rsid'].isin(snplist['alternate_id']).value_counts()

True    1255683
Name: rsid, dtype: int64

In [28]:
snplist['alternate_id'].isin(z['rsid']).value_counts()

True    1255683
Name: alternate_id, dtype: int64

# Run LDstore

In [ ]:
~/ldstore_v2.0_x86_64/./ldstore_v2.0_x86_64 --in-files /mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/111822_LDstore_files/master_file --write-bcor --read-only-bgen

[MetaKernelApp] ERROR | KeyboardInterrupt caught in kernel.


In [ ]:
[global]
# Working directory: change accordingly
parameter: cwd = path
# Path to bgen or plink files
parameter: masterfile = path
#Path to the rsid file
parameter: rsid_file = path('.')
parameter: mem = '80G'
parameter: numThreads = 4
parameter: job_size = 1

In [ ]:
# Creation of the z file
import glob, os
import pandas as pd
# assign directory
regions=glob.glob("/mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/113022_regions_bgen/ukb_imp_chr*.variants")
mfi_files=glob.glob("/mnt/vast/hpc/csg/UKBiobank_Yale_transfer/ukb39554_imputeddataset/ukb_mfi_chr*_v3.txt")
for filename in regions:
    print(filename)
    basename=os.path.basename(filename)
    sp = basename.split("_")
    chrom = [c for c in sp if "chr" in c][0]
    print(basename)
    fullregion = pd.read_csv(filename,header=0,sep="\t", skiprows=1, skipfooter=1, names=['alternate_ids', 'rsid', 'chromosome', 'position', 'number_of_alleles', 'allele1', 'allele2'])
    print('The number of variants in the region is:',fullregion.shape[0])
    mfi_file = [file for file in mfi_files if chrom in file][0]
    chrom_file = pd.read_csv(mfi_file,header=None,sep="\t", names=["alternate_id", "rsid", "position", "allele1", "allele2", "maf", "minor_allele", "info_score"])
    print('The number of variants in',mfi_file,'is:',chrom_file.shape[0])
    subset_region=chrom_file.join(fullregion.set_index(['alternate_ids', 'rsid', 'position', 'allele1', 'allele2']), on=['alternate_id', 'rsid', 'position', 'allele1', 'allele2'], how='right')
    maf=0.01
    subset_maf=subset_region[subset_region['maf'] > maf]
    print('The number of variants in the maf',maf,'filtered z_file is:',subset_maf.shape[0])
    subset_maf[["rsid","chromosome", "position", "allele1", "allele2", "alternate_ids"]].to_csv('/mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/111822_LDstore_files/z_files/' + basename + ".z" , sep='\t', index=False, header=True)

In [ ]:
# Creation of the masterfile

In [ ]:
# Perform LD calculation in LDStore
[default]
input: masterfile
task: trunk_workers = 1, trunk_size = job_size, walltime = '48h', mem = mem, cores = numThreads, tags = f'{_input:bn}'
bash: expand= "${ }", stderr = f'{_input}.stderr', stdout = f'{_input}.stdout'   
    ~/ldstore_v2.0_x86_64/./ldstore_v2.0_x86_64  \
    --in-files ${_input} \
    --write-bcor \
    --read-only-bgen \
    --n-threads ${numThreads} 

In [21]:
## Set the bash variables 
cwd=/mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/111822_LDstore_files/
ldstore_sbatch=$cwd/ldstore_test_$(date +"%Y-%m-%d").sbatch
masterfile=/mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/111822_LDstore_files/master_file_chr22
jobsize=1
ldstore_sos=~/project/UKBB_GWAS_dev/workflow/111722_LDstore.ipynb
tpl_file=~/project/bioworkflows/admin/csg.yml
mem='80G'
job_size=1
numThreads=4
#rsid_file=/mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/111822_LDstore_files/chr22_snplist.snplist

ldstore_args="""default 
    --cwd $cwd
    --masterfile $masterfile
    --numThreads $numThreads 
    --mem $mem 
    --job_size $job_size
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $ldstore_sos \
    --to-script $ldstore_sbatch \
    --args "$ldstore_args"

INFO: Running csg: Configuration for Columbia csg partition cluster
INFO: csg is completed.
INFO: csg output:   /mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/111822_LDstore_files/ldstore_test_2022-11-28.sbatch
INFO: Workflow csg (ID=w4e35c51f1070ff19) is executed successfully with 1 completed step.



In [37]:
export PATH=$HOME/miniconda3/bin:$PATH
module load Singularity/3.5.3
sos run /home/dmc2245/project/UKBB_GWAS_dev/workflow/111722_LDstore.ipynb \
    default \
    --cwd /mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/111822_LDstore_files/\
    --masterfile /mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/111822_LDstore_files/masterfile_chr22_alternateid\
    --numThreads 4 \
    --mem 80G     --job_size 1 \
    -c /home/dmc2245/project/bioworkflows/admin/csg.yml \
    -q csg -s force \
    &> /mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/111822_LDstore_files/ldstore_test_2022-11-21.log

In [ ]:
#master file with alternate_id chr22
z;bgen;bgi;bcor;ld;n_samples
/mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/111822_LDstore_files/ukb39554_imputeddataset/chr22_test.z;/mnt/vast/hpc/csg/UKBiobank_Yale_transfer/ukb39554_imputeddataset/ukb_imp_chr22_v3.bgen;/mnt/vast/hpc/csg/UKBiobank_Yale_transfer/ukb39554_imputeddataset/ukb_imp_chr22_v3.bgen.bgi;/mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/111822_LDstore_files/ukb39554_imputeddataset/ukb_imp_chr22_v3.bcor;/mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/111822_LDstore_files/ukb39554_imputeddataset/ukb_imp_chr22_v3.ld;487409

## Error troubleshooting

* Using the alternate_id from the bgen file creates even more errors
* Running the chr22 with the `--rsid` and `--write-text` option finds duplicated rsids which stops from running
* Issue is definately arising from the rsid which is used in the bgen file and not the alternate_id which is not duplicated

### Find duplicated rsids

In [9]:
import pandas as pd
file= '/mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/111822_LDstore_files/chr22_snplist.snplist'
rsid = pd.read_csv(file,header=None,sep=" ",names=["rsid"])

In [10]:
rsid

,rsid
0,rs587697622
1,rs587755077
2,rs587654921
3,rs587712275
4,22:16050435_T_C
...,...
1255678,rs546768261
1255679,rs568168135
1255680,rs199560686
1255681,rs556942236


In [11]:
seen = set()
dupes = [x for x in rsid if x in seen or seen.add(x)] 

In [15]:
duplicate = rsid[rsid.duplicated()]

In [16]:
duplicate

,rsid
1883,rs566228871
3620,rs567226117
4135,rs551161431
4711,rs530623904
5771,rs576124434
...,...
1253301,rs563631230
1253425,rs545727410
1253599,rs568699961
1254802,rs563437836


### Find the list of variants present in the bgen file

In [3]:
bgenix -g /mnt/vast/hpc/csg/UKBiobank_Yale_transfer/ukb39554_imputeddataset/ukb_imp_chr22_v3.bgen -list > /mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/111822_LDstore_files/chr22_variants.txt


Welcome to bgenix
(version: 1.1.7, revision )

(C) 2009-2017 University of Oxford

Building query                                              :  (1255683/?,0.6s,2169133.9/s)

Thank you for using bgenix.



In [ ]:
bgenix -g /mnt/vast/hpc/csg/UKBiobank_Yale_transfer/ukb39554_imputeddataset/ukb_imp_chr22_v3.bgen -list > /mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/111822_LDstore_files/chr22_variants.txt \
| grep rs999224

/home/dmc2245/miniconda3/lib/python3.8/site-packages/sos_notebook/kernel.py:1334: DeprecationWarning: Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  msg['msg_id'] = self._parent_header['header']['msg_id']
[MetaKernelApp] ERROR | KeyboardInterrupt caught in kernel.


In [5]:
#This is the numer of rows in this file. However it does not reflect the number of variants since this is multiallelic
cat /mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/111822_LDstore_files/chr22_variants.txt \
| grep "rs999224"

rs999224	rs999224	22	27834500	2	A	G



* The error arises since there are some variants in the mfi file that are not present in the bgen file 

* Next step is to remove the variants not present and re-run the analysis

* Error was on the z file with some variants not having a correct chromosome column

In [9]:
cat /mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/111822_LDstore_files/master_file_chr22.stdout | grep "Error" | awk -F"'" '{print $2 }'\
> /mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/111822_LDstore_files/rsid_not_found.txt

In [11]:
cat /mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/111822_LDstore_files/chr22_variants.txt | grep -xF /mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/111822_LDstore_files/rsid_not_found.txt

In [1]:
cd /mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/111822_LDstore_files/
#grep -f rsid_not_found.txt chr22_variants.txt

/home/dmc2245/miniconda3/lib/python3.8/site-packages/sos_notebook/kernel.py:1334: DeprecationWarning: Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  msg['msg_id'] = self._parent_header['header']['msg_id']


In [2]:
cat rsid_not_found.txt |wc -l

11072


In [7]:
sort rsid_not_found.txt | uniq | wc -l

11072


In [3]:
# 11117 -  45 variants have same rsid but different alleles
grep -wf rsid_not_found.txt chr22_variants.txt | wc -l 

rs62224618	rs62224618	22	16057417	2	T	C
rs116911124	rs79847867	22	16495833	2	C	A
rs117578132	rs117578132	22	16595552	2	C	A
rs131533	rs131533	22	16870425	2	C	T
rs140380	rs140380	22	16880098	2	A	G
rs1807483	rs1807483	22	16888577	2	A	G
rs13053962	rs13053962	22	16890136	2	G	A
rs5992618	rs5992618	22	16915685	2	G	A
rs79754891	rs79754891	22	16952830	2	C	G
rs73159809	rs73159809	22	17054795	2	G	A
rs5747940	rs5747940	22	17056415	2	T	C
rs5746647	rs5746647	22	17057138	2	G	T
rs62228720	rs4819760	22	17057597	2	G	A
rs112574411	rs112574411	22	17063176	2	C	T
rs5993499	rs5993499	22	17068748	2	C	T
rs13053626	rs13053626	22	17070109	2	A	G
rs139948519	rs139948519	22	17072347	2	C	T
rs200670934	rs200670934	22	17073119	2	C	T
rs2070503	rs2070503	22	17073475	2	C	T
rs111636940	rs111636940	22	17079911	2	A	G
rs5746677	rs5746677	22	17080190	2	G	C
rs7285553	rs7285553	22	17091201	2	C	A
rs111736353	rs111736353	22	17091621	2	A	T
rs16984366	rs16984366	22	17096864	2	T	C
rs9606009	rs9606009	22	17103078	2	C	G
rs117430084	rs

In [10]:
grep -wf rsid_not_found.txt /mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/111822_LDstore_files/ukb39554_imputeddataset/ukb_mfi_chr22_v3.txt.z \
> /mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/111822_LDstore_files/chr22_notfound.z

## New run with the block data

In [17]:
## Set the bash variables 
cwd=/mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/113022_regions_bgen
ldstore_sbatch=$cwd/ldstore_blocks_$(date +"%Y-%m-%d").sbatch
masterfile=/mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/113022_regions_bgen/chr22_region_test.masterfile
jobsize=1
ldstore_sos=~/project/UKBB_GWAS_dev/workflow/111722_LDstore.ipynb
tpl_file=~/project/bioworkflows/admin/csg.yml
mem='40G'
job_size=1
numThreads=4
rsid_file=/mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/111822_LDstore_files/ukb_imp_chr22_v3.22:16050408-17674295.variants.mafabove0.0

ldstore_args="""default 
    --cwd $cwd
    --masterfile $masterfile
    --numThreads $numThreads 
    --mem $mem 
    --rsid_file $rsid_file
    --job_size $job_size
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $ldstore_sos \
    --to-script $ldstore_sbatch \
    --args "$ldstore_args"

INFO: Running csg: Configuration for Columbia csg partition cluster
INFO: csg is completed.
INFO: csg output:   /mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/113022_regions_bgen/ldstore_blocks_2022-12-09.sbatch
INFO: Workflow csg (ID=w847595baef3d3429) is executed successfully with 1 completed step.



In [ ]:
#master file with a block of chr22
z;bgen;bgi;bcor;ld;n_samples
/mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/113022_regions_bgen/ukb_imp_chr22_v3.22:16050408-17674295.variants.z;/mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/113022_regions_bgen/ukb_imp_chr22_v3.22:16050408-17674295.bgen;/mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/113022_regions_bgen/ukb_imp_chr22_v3.22:16050408-17674295.bgen.bgi;/mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/113022_regions_bgen/ukb_imp_chr22_v3.22:16050408-17674295.bgen.bcor;/mnt/vast/hpc/csg/UKBiobank/results/pleiotropy_AD_ARHI/113022_regions_bgen/ukb_imp_chr22_v3.22:16050408-17674295.bgen.ld;487409